## Experiment 5 : Greedy Best First Search Algorithm

In [27]:
import heapq

def greedy_best_first_search(start, goal, graph, heuristic):
    # Initialize open list (priority queue) and visited set
    open_list = [(heuristic[start], start)]
    visited = set()

    # Dictionary to keep track of the path
    path = {start: None}

    while open_list:
        print('========================================================')
        print(f' Open List is : {open_list}')

        # Get node with lowest heuristic value
        _, current = heapq.heappop(open_list)
        print(f'\n Current Node explored is : {current} with Heuristic : {heuristic[current]} \n')

        # Check if goal node is reached
        if current == goal:
            return reconstruct_path(path, start, goal)

        # Mark current node as visited
        visited.add(current)

        # Explore neighbors of the current node
        neighbors = []
        for neighbor in graph[current]:
            if neighbor not in visited and neighbor not in [n[1] for n in open_list]:
                neighbors.append((heuristic[neighbor], neighbor))
                path[neighbor] = current  # Set the parent of the neighbor

        # Sort neighbors by heuristic value
        neighbors.sort(key=lambda x: x[0])

        # Add neighbors to open list
        for neighbor in neighbors:
            heapq.heappush(open_list, neighbor)
            print(f' Neighbor added: {neighbor[1]} with Heuristic : {neighbor[0]}')

        # Check if we're stuck (no unvisited neighbors with better heuristic)
        if not neighbors and (not open_list or heuristic[open_list[0][1]] >= heuristic[current]):
            return reconstruct_path(path, start, current)

    # No path found
    return reconstruct_path(path, start, current)

def reconstruct_path(path, start, end):
    reconstructed_path = []
    current = end
    while current is not None:
        reconstructed_path.append(current)
        current = path[current]
    reconstructed_path.reverse()
    return reconstructed_path

# Example graph represented as a dictionary
# Keys are nodes, values are lists of neighbors
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['I'],
    'E': [],
    'F': [],
    'G': ['H'],
    'H': []
}

# Example heuristic function represented as a dictionary
# Keys are nodes, values are estimated costs to goal
heuristic = {
    'A': 7,
    'B': 6,
    'C': 3,
    'D': 5,
    'E': 1,
    'F': 9,
    'G': 2,
    'H': 0,  # Heuristic value of goal node should be 0
    'I': 0
}

# Get user input for start and goal nodes
start = input('Enter the Start Node : ')
goal = input('Enter the Goal  Node : ')

# Perform Greedy Best-First Search
result_path = greedy_best_first_search(start, goal, graph, heuristic)

# Print the result
if result_path[-1] == goal:
    print(f' Goal node {goal} reached!')
else:
    print(f" No path found from {start} to {goal}. Search stopped at {result_path[-1]}")

print(' Final Path is : ', end='')
print(' --> '.join(result_path))

Enter the Start Node : A
Enter the Goal  Node : H
 Open List is : [(7, 'A')]

 Current Node explored is : A with Heuristic : 7 

 Neighbor added: C with Heuristic : 3
 Neighbor added: B with Heuristic : 6
 Open List is : [(3, 'C'), (6, 'B')]

 Current Node explored is : C with Heuristic : 3 

 Neighbor added: G with Heuristic : 2
 Neighbor added: F with Heuristic : 9
 Open List is : [(2, 'G'), (6, 'B'), (9, 'F')]

 Current Node explored is : G with Heuristic : 2 

 Neighbor added: H with Heuristic : 0
 Open List is : [(0, 'H'), (9, 'F'), (6, 'B')]

 Current Node explored is : H with Heuristic : 0 

 Goal node H reached!
 Final Path is : A --> C --> G --> H


### Experiment 6 : A* Search Algorithm

In [28]:
import heapq
from collections import defaultdict

def astar(start, goal, graph, heuristic):
    # Initialize open and closed lists
    open_list = [(0, start)]
    closed_list = set()

    # Initialize g_scores (cost from start to node) and parents (for path reconstruction)
    g_scores = defaultdict(lambda: float('inf'))
    g_scores[start] = 0
    parents = {}

    while open_list:
        print('==================================================')
        print(f'   Open List is : {open_list}')

        # Get node with lowest f_score (f_score = g_score + heuristic)
        f_score, current = heapq.heappop(open_list)
        print(f'   Current Node is : {current} with Score : {f_score}')

        # Check if goal node is reached
        if current == goal:
            print('\n    Goal Node Reached !!!\n')
            # Reconstruct and return the path
            path = []
            while current:
                path.append(current)
                current = parents.get(current)
            return path[::-1]  # Reverse the path to get start-to-goal order

        # Add current node to closed list (mark as explored)
        closed_list.add(current)

        # Explore neighbors of the current node
        for neighbor, cost in graph[current].items():
            # Ignore neighbors that have already been explored
            if neighbor in closed_list:
                continue

            # Calculate tentative g_score for this neighbor
            tentative_g_score = g_scores[current] + cost
            print(f'      Neighbor Added : {neighbor} with Total Cost : {tentative_g_score + heuristic[neighbor]}')
            # Check if we found a better path
            if tentative_g_score < g_scores[neighbor]:
                # Update parent and g_score for the neighbor
                parents[neighbor] = current
                g_scores[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic[neighbor]

                # Add neighbor to open list if not already in it
                if neighbor not in [n[1] for n in open_list]:
                    heapq.heappush(open_list, (f_score, neighbor))
                else:
                    # Update neighbor's position in open list
                    open_list = [(f, n) if n != neighbor else (f_score, neighbor) for f, n in open_list]
                    heapq.heapify(open_list)

        # Print current path for visualization
        tempPath = []
        curr = current
        while curr:
            tempPath.append(curr)
            curr = parents.get(curr)
        print('    Current Path is : ', end='')
        print(' --> '.join(tempPath[::-1]))

    # No path found
    return None

# Example graph and heuristic remain unchanged
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'D': 1, 'E': 2},
    'C': {'F': 5, 'G': 3},
    'D': {},
    'E': {},
    'F': {},
    'G': {'H': 2},
    'H': {},
    'I': {'J':4},
    'J': {}
}

heuristic = {
    'A': 7, 'B': 6, 'C': 3, 'D': 5,'E': 1,
    'F': 9, 'G': 2, 'H': 0, 'J': 2,'I': 0
}

# Get user input for start and goal nodes
start = input(' Enter the Start Node : ')
goal = input(' Enter the Goal  Node : ')

# Perform A* search
path = astar(start, goal, graph, heuristic)

# Print the final path
print('   Final Path is : ', end='')
print(' --> '.join(path) if path else "No path found")

 Enter the Start Node : A
 Enter the Goal  Node : H
   Open List is : [(0, 'A')]
   Current Node is : A with Score : 0
      Neighbor Added : B with Total Cost : 7
      Neighbor Added : C with Total Cost : 7
    Current Path is : A
   Open List is : [(7, 'B'), (7, 'C')]
   Current Node is : B with Score : 7
      Neighbor Added : D with Total Cost : 7
      Neighbor Added : E with Total Cost : 4
    Current Path is : A --> B
   Open List is : [(4, 'E'), (7, 'D'), (7, 'C')]
   Current Node is : E with Score : 4
    Current Path is : A --> B --> E
   Open List is : [(7, 'C'), (7, 'D')]
   Current Node is : C with Score : 7
      Neighbor Added : F with Total Cost : 18
      Neighbor Added : G with Total Cost : 9
    Current Path is : A --> C
   Open List is : [(7, 'D'), (18, 'F'), (9, 'G')]
   Current Node is : D with Score : 7
    Current Path is : A --> B --> D
   Open List is : [(9, 'G'), (18, 'F')]
   Current Node is : G with Score : 9
      Neighbor Added : H with Total Cost : 9
  